### Using multiple tree methods 
- Decesion Trees 
- Random Forest 
- Gradient Boosted trees

In [1]:
from pyspark.sql import SparkSession 

# creating a session 
spark = SparkSession.builder.appName('trees').getOrCreate()

22/06/06 21:08:35 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.8.100 instead (on interface wlp1s0)
22/06/06 21:08:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/06 21:08:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# using Pipline to keep things in order 
from pyspark.ml import Pipeline

# load the algorthims 
from pyspark.ml.classification import (RandomForestClassifier ,GBTClassifier , 
                                        DecisionTreeClassifier)


In [4]:
# loading data 

df = spark.read.csv('College.csv', inferSchema=True , header=True)

In [5]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
df.show(1)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
only showing top 1 row



In [7]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=[
    'Apps',
    'Accept',
    'Enroll',
    'Top10perc',
    'Top25perc',
    'F_Undergrad',
    'P_Undergrad',
    'Outstate',
    'Room_Board',
    'Books',
    'Personal',
    'PhD',
    'Terminal',
    'S_F_Ratio',
    'perc_alumni',
    'Expend',
    'Grad_Rate'
], outputCol='featurs')

In [10]:
output = assembler.transform(df)

In [11]:
# we want to predict "Private", but it is a string 
# we need to trnasform it to (0, 1)
from pyspark.ml.feature import StringIndexer 

indexer = StringIndexer(inputCol='Private' , outputCol= 'PrivateIndex')

In [12]:
output_fixed = indexer.fit(output).transform(output)

In [13]:
# check 
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- featurs: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [15]:
# selecting the needed columns only 

final_data = output_fixed.select('featurs' , 'PrivateIndex') 

In [16]:
final_data.show(1)

+--------------------+------------+
|             featurs|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
+--------------------+------------+
only showing top 1 row



In [17]:
# splitting 

train_data , test_data = final_data.randomSplit([0.7 , 0.3]) 

### Using the  three trees method 

In [20]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex' , featuresCol='featurs')
rfc = RandomForestClassifier(labelCol='PrivateIndex' , featuresCol='featurs')
gbt = GBTClassifier(labelCol='PrivateIndex' , featuresCol='featurs')

In [25]:
# the models 
dtc_model = dtc.fit(train_data) 
rfc_model = rfc.fit(train_data) 
gbt_model = gbt.fit(train_data) 

In [26]:
# transform to test data 
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [27]:
# Evaluate 
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [28]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [29]:
# printing the evaluations 

print('DTC')
print(my_binary_eval.evaluate(dtc_preds)) 
print('===') 
print('RFC') 
print(my_binary_eval.evaluate(rfc_preds)) 
print('===') 
print('GBT') 
print(my_binary_eval.evaluate(gbt_preds)) 


DTC
0.8485294117647059
===
RFC
0.9657983193277311
===
GBT
0.9363445378151259


In [30]:
# Another evaluating mertics 

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex",
                 predictionCol="prediction", metricName="accuracy")

In [32]:
dtc_acc = acc_evaluator.evaluate(dtc_preds)
rfc_acc = acc_evaluator.evaluate(rfc_preds)
gbt_acc = acc_evaluator.evaluate(gbt_preds)

In [33]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 90.00%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 90.42%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 89.58%


### Notes: 
- We can imrove the accuracy of RFC, GBT by increase the number of trees. 